## Deploy

In [12]:
# deploy model

# deploy model

import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import Model
from sagemaker.async_inference import AsyncInferenceConfig
import boto3
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# CONFIG
endpoint_name = "vocal-remover-sagemaker-inference-endpoint-v1"
region = 'us-east-1'
output_path = f's3://vocalremoverbucket.us.east.1.only/sagemaker_inferences/{endpoint_name}/outputs'
model_path = 's3://vocalremoverbucket.us.east.1.only/models/models_only_v2.tar.gz'

# Your specified role and git configuration
role = 'arn:aws:iam::975050117348:role/service-role/SageMaker-ExecutionRole-20240223T141256'
# it should be public(you can make it priv after this process), or you can give access permissions
git_config = {'repo': 'https://github.com/us/MVSEP-MDX23-AWS.git',
              'branch': 'v2.3',}

boto_session = boto3.Session(region_name=region)

# Create a SageMaker session using the boto3 session
sagemaker_session = sagemaker.Session(boto_session=boto_session)


# Specify the S3 output path for inference results
# output_path = f's3://vocalremoverbucket.us.east.1.only/{endpoint_name}/outputs/'
# vocal-remover-sagemaker-endpoint-v1

# Specify the notification configuration
notification_config = {
    'SuccessTopic': 'arn:aws:sns:us-east-1:975050117348:vr-success',
    'ErrorTopic': 'arn:aws:sns:us-east-1:975050117348:vr-failure'
}

# Create an AsyncInferenceConfig object
async_inference_config = AsyncInferenceConfig(
    output_path=output_path,
    max_concurrent_invocations_per_instance=1,  # adjust this based on your needs
    notification_config=notification_config,
    failure_path=output_path + 'failure/'  # optional, specify if you want a separate failure path
)


# Create a SageMaker PyTorchModel object
pytorch_model = PyTorchModel(model_data=model_path,
                             git_config=git_config,
                             role=role,
                             entry_point='inference.py',
                             source_dir='code',
                             framework_version='2.0.1',
                             py_version='py310',
                             sagemaker_session=sagemaker_session,
                             env={'MODEL_SERVER_TIMEOUT': '600',
                                  'MODEL_SERVER_RESPONSE_TIMEOUT': '600',
                                  'MODEL_SERVER_TIMEOUT_ENV': '600',
                                  'SAGEMAKER_MODEL_SERVER_TIMEOUT': '600',
                                  'DEFAULT_MODEL_SERVER_TIMEOUT':'600'
                                 }
                            )

# Deploy your model to an asynchronous inference endpoint
predictor = pytorch_model.deploy(
    instance_type='ml.g4dn.xlarge',
    initial_instance_count=1,
    endpoint_name=endpoint_name,
    async_inference_config=async_inference_config,  
    wait=False,  
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)


Cloning into '/tmp/tmpwz123pfo'...
Already on 'v2.3'


Your branch is up to date with 'origin/v2.3'.


## Helper func for reading prediction output

In [3]:
# helper funcs
import boto3
from urllib.parse import urlparse

def separate_bucket_and_path(s3_uri):
    parsed = urlparse(s3_uri)
    bucket = parsed.netloc
    path = parsed.path.lstrip('/')
    return bucket, path

def retrieve_s3_file_contents(s3_uri):
    """
    Retrieve and print the contents of a file from an S3 bucket.

    :param bucket_name: The name of the S3 bucket.
    :param object_key: The key of the object within the bucket.
    """
    parsed = urlparse(s3_uri)
    bucket = parsed.netloc
    path = parsed.path.lstrip('/')
#     print(bucket, "\n", path)
    # Initialize an S3 client
    s3_client = boto3.client('s3')

    try:
        # Retrieve the object
        response = s3_client.get_object(Bucket=bucket, Key=path)
        
        # Read the object's content
        content = response['Body'].read().decode('utf-8')
        
        print("File contents:", content)
    
    except s3_client.exceptions.NoSuchBucket:
        print("The specified bucket does not exist.")
    except s3_client.exceptions.NoSuchKey:
        print("The specified key does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")


## Split

In [13]:
from sagemaker.predictor import Predictor
from sagemaker.pytorch.model import PyTorchPredictor

from sagemaker.predictor_async import AsyncPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import boto3
import sagemaker
import boto3
import json

import boto3
import json

import base64
region = 'us-east-1'
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)

input_data = {
    "s3_audio_path": "s3://vocalremoverbucket.us.east.1.only/input-path/song.mp3",
    "options": {
        "s3_output_bucket": "vocalremoverbucket.us.east.1.only",
        "s3_sagemaker_outputs_folder_path": f"sagemaker-{endpoint_name}-default-outputs",
        "overlap_demucs": 0.1,
        "overlap_VOCFT": 0.1,
        "overlap_VitLarge": 1,
        "overlap_InstVoc": 1,
        "weight_InstVoc": 8,
        "weight_VOCFT": 1,
        "weight_VitLarge": 5,
        "large_gpu": True,
        "BigShifts": 1,
        "vocals_only": False,
        "use_VOCFT": False,
        "output_format": "FLOAT",
    }
}

# Create a predictor
predictor2 = PyTorchPredictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    sagemaker_session=sagemaker_session
)
# you can give random path, it basically uploads input_data json to s3 for sagemaker easy read! so it's not's matter
input_path='s3://vocalremoverbucket.us.east.1.only/default-input-path/input-data.json'
async_predictor = AsyncPredictor(predictor2)

# Multi request
# ress = []
# for i in range(20):
#     res = async_predictor.predict_async(data=input_data, input_path=input_path)
#     ress.append(res)
# print(ress)
# Process the prediction result as needed
res = async_predictor.predict_async(data=input_data, input_path=input_path)
print(res.__dict__)

{'predictor_async': <sagemaker.predictor_async.AsyncPredictor object at 0x7fc206302920>, 'output_path': 's3://vocalremoverbucket.us.east.1.only/sagemaker_inferences/vocal-remover-sagemaker-inference-endpoint-v1/outputs/1325527b-0250-4e20-8880-90e2b03edc94.out', '_result': None, 'failure_path': 's3://vocalremoverbucket.us.east.1.only/sagemaker_inferences/vocal-remover-sagemaker-inference-endpoint-v1/outputsfailure/1325527b-0250-4e20-8880-90e2b03edc94-error.out'}


## Retrieve the outputs if exists

In [27]:
# answers = pred.__dict__
answers = res.__dict__

print("SUCCESS")
retrieve_s3_file_contents(answers['output_path'])
print("FAIL")
retrieve_s3_file_contents(answers['failure_path'])
# res.__dict__


SUCCESS
File contents: {
  "out_paths": [
    "s3://vocalremoverbucket.us.east.1.only/sagemaker-vocal-remover-sagemaker-inference-endpoint-v1-default-outputs/2024-03-08-10-56-27-2a70bbdb-c1b6-4289-b14f-32fec68837c5/bass.wav",
    "s3://vocalremoverbucket.us.east.1.only/sagemaker-vocal-remover-sagemaker-inference-endpoint-v1-default-outputs/2024-03-08-10-56-27-2a70bbdb-c1b6-4289-b14f-32fec68837c5/drums.wav",
    "s3://vocalremoverbucket.us.east.1.only/sagemaker-vocal-remover-sagemaker-inference-endpoint-v1-default-outputs/2024-03-08-10-56-27-2a70bbdb-c1b6-4289-b14f-32fec68837c5/other.wav",
    "s3://vocalremoverbucket.us.east.1.only/sagemaker-vocal-remover-sagemaker-inference-endpoint-v1-default-outputs/2024-03-08-10-56-27-2a70bbdb-c1b6-4289-b14f-32fec68837c5/vocals.wav"
  ]
}
FAIL
The specified key does not exist.
